In [2]:
import json
import requests as r
from rauth.service import OAuth2Service
import pandas as pd
import numpy as np
import random

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
barentswatch = OAuth2Service(
    name='barentswatch',
    client_id='zryan.rzgar.hassan.mustafa@digdir.no:CampSommer',
    client_secret='ZryanKristian',
    authorize_url='https://id.barentswatch.no/connect/authorize',
    access_token_url='https://id.barentswatch.no/connect/token',
    base_url='https://www.barentswatch.no/bwapi/')

data = {
    'scope': 'api',
    'grant_type': 'client_credentials',
    }

session = barentswatch.get_auth_session(data=data, decoder=json.loads)

In [6]:
res = r.get(
    'https://www.barentswatch.no/bwapi/v1/geodata/fishhealth/localitieswithsalmonoids',
    headers = { 'Authorization' : 'Bearer ' + session.access_token }
)

df = pd.DataFrame(res.json())

dfescape = pd.DataFrame()
locnr = df['localityNo'].tolist()
    

In [10]:

for i in locnr:
    res = r.get(
        'https://www.barentswatch.no/bwapi/v1/geodata/fishhealth/locality/'+str(locnr)+'/escape/2022',
        headers = { 'Authorization' : 'Bearer ' + session.access_token }
    )
    
    if res.json()['data']:
        dict_new = res.json()['data'][0]['escapes'][0]
        dfescape = dfescape.append(dict_new, ignore_index=True)   
    else:
        dict_new = res.json()
        dfescape = dfescape.append(dict_new, ignore_index=True) 
        

/var/folders/mz/r6r6r9cx47vdpt1c1jz6gv7r0000gn/T/ipykernel_83239/1551850208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfescape = dfescape.append(dict_new, ignore_index=True)
/var/folders/mz/r6r6r9cx47vdpt1c1jz6gv7r0000gn/T/ipykernel_83239/1551850208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfescape = dfescape.append(dict_new, ignore_index=True)
/var/folders/mz/r6r6r9cx47vdpt1c1jz6gv7r0000gn/T/ipykernel_83239/1551850208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfescape = dfescape.append(dict_new, ignore_index=True)
/var/folders/mz/r6r6r9cx47vdpt1c1jz6gv7r0000gn/T/ipykernel_83239/1551850208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [8]:
dfescape = dfescape.drop('data', 1)

/var/folders/mz/r6r6r9cx47vdpt1c1jz6gv7r0000gn/T/ipykernel_83239/1588913603.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfescape = dfescape.drop('data', 1)


In [11]:
dfescape

,localityNo,year,data
0,30096,2022,NaN
1,12765,2022,NaN
2,10665,2022,NaN
3,13823,2022,NaN
4,33437,2022,NaN
5,30196,2022,NaN
6,11934,2022,NaN
7,35297,2022,NaN
8,12084,2022,NaN
9,30977,2022,NaN


In [19]:
dflus = pd.DataFrame()
locnr = df['localityNo'].tolist()
for i in locnr:
    res = r.get(
        'https://www.barentswatch.no/bwapi/v1/geodata/fishhealth/locality/'+str(i)+'/avgfemalelice/2022',
        headers = { 'Authorization' : 'Bearer ' + session.access_token }
    )
    
    #print(res.json())
    
    #if res.json()['data']:
    #    dict_new = res.json()['data']
    #    print(dict_new)
    #    print('-----')
    #    dflus = dflus.append(dict_new, ignore_index=True)
    #else:
    dict_new = res.json()
    dflus = dflus.append(dict_new, ignore_index=True)
        

In [20]:
dflus = dflus.drop('type', 1)

In [21]:
dflus

,localityNo,year,data
0,30096,2022,[]
1,12765,2022,[]
2,10665,2022,[]
3,13823,2022,[]
4,33437,2022,[]
...,...,...,...
1915,10805,2022,[]
1916,20176,2022,[]
1917,11338,2022,[]
1918,29417,2022,[]


In [22]:
dfas = pd.read_csv('as.csv', sep = ';')
dfas['LOK_KAP'] = dfas['LOK_KAP'].str.replace(',','.')

In [23]:
#dfas.loc[dfas['LOK_NR'] == 10362]

In [24]:
totalgood = 0
totalfail = 0
locnrmedas = []
for i in locnr:
    if len(dfas.loc[dfas['LOK_NR'] == i]) == 1:
        totalgood += 1
        locnrmedas.append(i)
    else:
        totalfail +=1

In [25]:
listemedlocnreksempel = dfas.loc[dfas['ORG.NR/PERS.NR'] == 918858741]['LOK_NR'].tolist()
listemedlocnreksempel

[45030, 45031]

In [26]:
dfdead = pd.DataFrame()

In [27]:
deadlighet = []
for i in locnrmedas:

    enhet = dfas.loc[dfas['LOK_NR'] == i]['LOK_ENHET'].values[0]
    kapasitet = dfas.loc[dfas['LOK_NR'] == i]['LOK_KAP'].values[0]
    
    if enhet == 'STK':
        konvertert = int(int(dfas.loc[dfas['LOK_NR'] == i]['LOK_KAP'].values[0])/5)
        dfas['LOK_KAP'][dfas.loc[dfas['LOK_NR']==i].index[0]] = konvertert
        dfas['LOK_ENHET'][dfas.loc[dfas['LOK_NR']==i].index[0]] = 'TN'
    elif enhet == 'KG':
        konvertert = int(int(dfas.loc[dfas['LOK_NR'] == i]['LOK_KAP'].values[0])/1000)
        dfas['LOK_KAP'][dfas.loc[dfas['LOK_NR']==i].index[0]] = konvertert
        dfas['LOK_ENHET'][dfas.loc[dfas['LOK_NR']==i].index[0]] = 'TN'
    elif enhet == 'M3':
        konvertert = int(int(dfas.loc[dfas['LOK_NR'] == i]['LOK_KAP'].values[0])*0.005)
        dfas['LOK_KAP'][dfas.loc[dfas['LOK_NR']==i].index[0]] = konvertert
        dfas['LOK_ENHET'][dfas.loc[dfas['LOK_NR']==i].index[0]] = 'TN'
    
    
    deadlighet.append(int(float(dfas.loc[dfas['LOK_NR'] == i]['LOK_KAP'].values[0])*random.uniform(12.5, 17.5)/100))
    
dictdead = {'LOK_NR':locnrmedas,'Year': 2022,'Deadlighet':deadlighet, 'Enhet': 'TN'}
dfdead = pd.DataFrame(dictdead)
dfdead

/var/folders/lh/9jrcxwxs6_g4dk6jtpgbcbnw0000gn/T/ipykernel_14801/1744968731.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfas['LOK_KAP'][dfas.loc[dfas['LOK_NR']==i].index[0]] = konvertert
/var/folders/lh/9jrcxwxs6_g4dk6jtpgbcbnw0000gn/T/ipykernel_14801/1744968731.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfas['LOK_ENHET'][dfas.loc[dfas['LOK_NR']==i].index[0]] = 'TN'
/var/folders/lh/9jrcxwxs6_g4dk6jtpgbcbnw0000gn/T/ipykernel_14801/1744968731.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

,LOK_NR,Year,Deadlighet,Enhet
0,10665,2022,0,TN
1,13823,2022,65181,TN
2,33437,2022,439,TN
3,30196,2022,15,TN
4,35297,2022,264,TN
...,...,...,...,...
1383,24175,2022,624,TN
1384,20176,2022,84,TN
1385,11338,2022,975,TN
1386,29417,2022,239,TN
